In [1]:
import json
from pyspark import SparkConf,SparkContext
import sys
import time

In [2]:
import os
from graphframes import *
from pyspark.sql.functions import col, lit, when
from pyspark.sql import SQLContext

In [3]:
t_start = time.time()

In [4]:
threshold = 7
input_file = 'ub_sample_data.csv'
output_file = 'task1'
#input_file = sys.argv[1]

In [5]:
os.environ["PYSPARK_SUBMIT_ARGS"] = ("--packages graphframes:graphframes:0.6.0-spark2.3-s_2.11 pyspark-shell")

In [6]:
conf = SparkConf().setAppName("inf553")
conf.set('spark.executor.memory', '4G')
conf.set('spark.driver.memory', '4G')

In [7]:
sc = SparkContext(conf=conf)

In [8]:
sqlContext = SQLContext(sc)

In [9]:
def graph_construction(chunk):
    user = chunk[0]
    business = chunk[1]
    result = []
    for key in user_dict:
        if key != user:
            if len(business&user_dict[key]) >= threshold:
                result.append((user,key))
    return result

In [10]:
input_data = sc.textFile(input_file)
first = input_data.first()
user_business = input_data.filter(lambda x: x != first).map(lambda s: s.split(','))

In [11]:
user_business_set = user_business.groupByKey().map(lambda t: (t[0], set(t[1]))).cache()

In [12]:
user_dict = dict(user_business_set.collect())

In [13]:
connect = user_business_set.flatMap(graph_construction)

In [14]:
node = connect.map(lambda t: (t[0],)).distinct()

In [15]:
vertices = sqlContext.createDataFrame(node, schema=['id'])

In [16]:
edges = sqlContext.createDataFrame(connect, schema=['src','dst'])

In [17]:
g = GraphFrame(vertices, edges)

In [18]:
result = g.labelPropagation(maxIter=5)

In [19]:
communities = result.rdd.map(lambda row: (row['label'], row['id'])).groupByKey().map(lambda t: (t[0], sorted(list(t[1])))) \
    .sortBy(lambda t:(len(t[1]),t[1][0])).collect()

In [20]:
with open(output_file, "w") as f:
    for community in communities:
        f.write(str(community[1])[1:-1] + '\n')
    

In [21]:
t_end = time.time()
print('Duration:', t_end-t_start)

Duration: 377.71437883377075
